In [1]:
# Importing all the necessary libraries
import numpy as np
import pandas as pd
# Importing all the necessary libraries
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.metrics as sm
from sklearn.metrics import mean_absolute_error
import time

In [2]:
# Importing the csv data files 
movies_metadata = pd.read_csv('metadata_integrated.csv',error_bad_lines=False, warn_bad_lines=True)
train_df = pd.read_csv('train_df.csv',error_bad_lines=False, warn_bad_lines=True)
test_df = pd.read_csv('test_df.csv',error_bad_lines=False, warn_bad_lines=True)

# Creating a new attribute by concating the title with the movieId to make the tiles unique
train_df['concat_title'] = train_df['title'].str.lower().str.replace(' ','') + train_df['movieId'].astype(str)
test_df['concat_title'] = test_df['title'].str.lower().str.replace(' ','') + test_df['movieId'].astype(str)


### Data Transformations

In [3]:
content_recom_df = movies_metadata[['movieId', 'overview','genres','title', 'production_companies', 'collectionName']]

In [4]:
content_recom_df['genres'] = content_recom_df['genres'].str.replace(',',' ').str.lower()
content_recom_df['concat_title'] = content_recom_df['title'].str.lower().str.replace(' ','') + content_recom_df['movieId'].astype(str)
content_recom_df['overview'] = content_recom_df['overview'].str.lower()
content_recom_df['collectionName'] = content_recom_df['collectionName'].str.replace(' ','').str.lower()
content_recom_df['production_companies'] = content_recom_df['production_companies'].str.replace(' ','').str.lower().str.replace(',',' ')

<ipython-input-4-6bdd9f41f4fc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_recom_df['genres'] = content_recom_df['genres'].str.replace(',',' ').str.lower()
<ipython-input-4-6bdd9f41f4fc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_recom_df['concat_title'] = content_recom_df['title'].str.lower().str.replace(' ','') + content_recom_df['movieId'].astype(str)
<ipython-input-4-6bdd9f41f4fc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

##  Model- using title, genre, overview, collectionName and production_company

In [5]:
# Create the corpus of data from the metadata of movies
content_recom_df['corpus'] = content_recom_df['title'].str.lower() + ' ' + content_recom_df['genres'] + ' ' + content_recom_df['overview'] + ' ' + content_recom_df['collectionName'] + ' ' + content_recom_df['production_companies'] 
content_recom_df['corpus'] = content_recom_df['corpus'].str.replace('[^\w\s]','') 
content_recom_df['corpus'] = content_recom_df['corpus'].str.replace('unknown','') 

<ipython-input-5-bbd4b19174a8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_recom_df['corpus'] = content_recom_df['title'].str.lower() + ' ' + content_recom_df['genres'] + ' ' + content_recom_df['overview'] + ' ' + content_recom_df['collectionName'] + ' ' + content_recom_df['production_companies']
<ipython-input-5-bbd4b19174a8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_recom_df['corpus'] = content_recom_df['corpus'].str.replace('[^\w\s]','')
<ipython-input-5-bbd4b19174a8>:4: Setti

In [6]:
# Reset index of your df and construct reverse mapping again
content_recom_df = content_recom_df.reset_index()
indices = pd.Series(content_recom_df.index, index=content_recom_df['concat_title'])

In [7]:
#Define a TF-IDF Vectorizer Object. Removing all english stopwords
tfidf = TfidfVectorizer(stop_words='english')


#Constructing the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(content_recom_df['corpus'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(9042, 43175)

In [8]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim_tfidf = linear_kernel(tfidf_matrix, tfidf_matrix)

In [9]:
# Getting the unique list of useres in the test dataset
users= test_df.userId.unique()

In [10]:
# This is the main function which predicts the ratings for the movies
def predit_content_rating(cosine_sim):
    users= test_df.userId.unique()
    for k in range(len(users)):
        for i in range(len(test_df.loc[(test_df['userId'] == users[k])])):
            temp_train_df = train_df.loc[(train_df['userId'] == users[k])]
            temp_test_df = test_df.loc[(test_df['userId'] == users[k])]
            sum_weight = 0
            rating_weight = 0
            prediction = 0
            title_idx = indices[temp_test_df.iloc[i].concat_title]
            ## Changed this line to cosine_sim2
            sim_scores = list(enumerate(cosine_sim[title_idx]))
            for j in range(len(temp_train_df)):
                idx = indices[temp_train_df.iloc[j].concat_title]
                weight = sim_scores[idx][1]
                rating_weight += (weight * (temp_train_df.iloc[j].rating))
                sum_weight += weight
            prediction = (rating_weight/sum_weight)
            test_df.loc[(test_df['userId'] == users[k]) & (test_df['title'] == temp_test_df.iloc[i].title),'predict'] = prediction

##  Corpus using - title, genre, overview, collectionName, production_companies

## Performance Evaluation using TF-IDF

In [11]:
# Model predicting the ratings of the movies using TF-IDF as feature engineering
predit_content_rating(cosine_sim_tfidf)

In [12]:
# The actual rating of the movies by the user
y_test = test_df['rating'].values
# The predicted rating of the movies
y_test_pred = test_df['predict'].values

In [13]:
# Root Mean Square Error of the model
rms = round(sqrt(mean_squared_error(y_test, y_test_pred )), 2)
print('Root Mean Square Error:',rms)

Root Mean Square Error: 0.95


In [14]:
# Mean Absolute Error of the model
mse = round(mean_absolute_error(y_test, y_test_pred), 2)
print('Mean Absolute Error:',mse)

Mean Absolute Error: 0.74


##  Corpus using - title, genre, overview

In [15]:
content_recom_df['corpus'] =  content_recom_df['genres'] + ' ' + content_recom_df['overview']  
content_recom_df['corpus'] = content_recom_df['corpus'].str.replace('[^\w\s]','') 
content_recom_df['corpus'] = content_recom_df['corpus'].str.replace('unknown','') 

In [16]:
#Define a TF-IDF Vectorizer Object. Removing all english stopwords
tfidf = TfidfVectorizer(stop_words='english')


#Constructing the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(content_recom_df['corpus'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(9042, 33717)

In [17]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim_tfidf = linear_kernel(tfidf_matrix, tfidf_matrix)

In [18]:
# Model predicting the ratings of the movies using TF-IDF as feature engineering
predit_content_rating(cosine_sim_tfidf)

### Performance Evaluation on the test data

In [19]:
global_mean_rating = np.average(train_df.rating.values)

In [20]:
global_mean_rating

3.5399152683169963

In [21]:
test_df = test_df.fillna(global_mean_rating)

In [22]:
# The actual rating of the movies by the user
y_test = test_df['rating'].values
# The predicted rating of the movies
y_test_pred = test_df['predict'].values

In [23]:
# Root Mean Square Error of the model
rms = round(sqrt(mean_squared_error(y_test, y_test_pred )), 2)
print('Root Mean Square Error:',rms)

Root Mean Square Error: 0.96


In [24]:
# Mean Absolute Error of the model
mse = round(mean_absolute_error(y_test, y_test_pred), 2)
print('Mean Absolute Error:',mse)

Mean Absolute Error: 0.74


### The two model gives the same accuracy so I have decided to use the model which uses overview and genre features

In [25]:
#Exporting the predicted values
test_df.to_csv("predicted_content_based_test_df.csv", index=False)

### Predicting the ratings in the train dataset

In [26]:
train_df = train_df[['userId','movieId','rating']]
test_df1 = train_df[['userId','movieId','rating']]

In [27]:
# Reset index of your df and construct reverse mapping again
content_recom_df = content_recom_df.reset_index()
indices = pd.Series(content_recom_df.index, index=content_recom_df['movieId'])

In [28]:
start = time.time()
users = test_df1.userId.unique()
k = 36 # Setting the value for the most similar movies number
start = time.time()
for i in range(len(users)):
    start1 = time.time()
    rated_movies = train_df.loc[(train_df['userId'] == users[i])].movieId.values
    unrated_movies = test_df1.loc[(test_df1['userId'] == users[i])].movieId.values
    for movie in rated_movies:
        idx = indices[movie]
        sim_scores = list(enumerate(cosine_sim_tfidf[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:k]
        movie_indices = [i[0] for i in sim_scores]
        movieIds = movies_metadata['movieId'].iloc[movie_indices].values    
        unrated_movies = list(set(movieIds).intersection(unrated_movies))
        if len(unrated_movies) > 0:
            for movie in unrated_movies:
                idx = indices[movie]
                sim_scores = list(enumerate(cosine_sim_tfidf[idx]))
                sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
                sim_scores = sim_scores[1:k]
                movieIds = movies_metadata['movieId'].iloc[movie_indices].values  
                movie_indices = [i[0] for i in sim_scores]
                
                movieIds = movies_metadata['movieId'].iloc[movie_indices].values  
                unrated_movies = list(set(movieIds).intersection(rated_movies))
                if len(unrated_movies)> 0:
                    weight_movies = [i[1] for i in sim_scores]
                    temp = pd.DataFrame()
                    temp2 = pd.DataFrame()
                    merged_rating_df = pd.DataFrame()
                    temp['movieId'] = movies_metadata['movieId'].iloc[movie_indices]
                    temp['weights'] = weight_movies
                    temp2['movieId'] = unrated_movies 
                    merged_rating_df = pd.merge(temp, temp2, how='inner', left_on=['movieId'], right_on = ['movieId'])
                    merged_rating_df = pd.merge(train_df.loc[(train_df['userId'] == users[i])], merged_rating_df , how='inner', left_on=['movieId'], right_on = ['movieId'])
                    merged_rating_df["weights*rating"] = merged_rating_df["weights"] * merged_rating_df["rating"]
                    weights = merged_rating_df['weights'].values
                    weights_rating = merged_rating_df['weights*rating'].values
                    prediction = sum(weights_rating)/(sum(weights))
                    test_df1.loc[(test_df1['userId'] == users[i]) & (test_df1['movieId'] == movie),'prediction'] = prediction     
    end1 = time.time()
    print('User : ', users[i])
    print(end1 - start1)  
end = time.time()
print(end - start)       

User :  1
0.08293747901916504
User :  7
0.4227604866027832
User :  31
0.42953968048095703
User :  32
0.2888350486755371
User :  36
0.5911593437194824
User :  39
0.33779168128967285
User :  73
8.641838550567627
User :  88
1.3848116397857666
User :  96
0.38542723655700684
User :  110
0.6157486438751221
User :  111
1.9302151203155518
User :  150
2.4913196563720703
User :  161
0.4757258892059326
User :  165
2.5389225482940674
User :  186
0.19088983535766602
User :  242
2.1906070709228516
User :  254
0.7432918548583984
User :  288
0.38979101181030273
User :  310
0.0779721736907959
User :  311
5.5195465087890625
User :  325
0.11093664169311523
User :  338
0.2848362922668457
User :  341
0.09194707870483398
User :  386
0.6644337177276611
User :  394
1.048396348953247
User :  452
7.372499465942383
User :  457
3.717576026916504
User :  461
3.6933352947235107
User :  468
6.85551905632019
User :  485
0.12896108627319336
User :  487
0.7931334972381592
User :  496
0.684748649597168
User :  511
0.133

User :  5
0.49883508682250977
User :  9
0.20803594589233398
User :  190
0.3268115520477295
User :  193
0.35002779960632324
User :  226
0.16621041297912598
User :  304
0.6156322956085205
User :  381
0.5693275928497314
User :  412
1.1339242458343506
User :  480
2.0637753009796143
User :  557
0.5301058292388916
User :  595
0.9181010723114014
User :  611
0.15194272994995117
User :  656
0.7011568546295166
User :  75
0.733497142791748
User :  157
1.714280366897583
User :  167
0.09896063804626465
User :  199
2.167992353439331
User :  355
1.6690311431884766
User :  395
0.09296321868896484
User :  482
0.27089953422546387
User :  544
1.3870079517364502
User :  599
1.0279929637908936
User :  626
0.79599928855896
User :  68
0.6589603424072266
User :  282
0.6381967067718506
User :  76
0.08300471305847168
User :  83
0.8269093036651611
User :  125
1.0650920867919922
User :  134
1.6378076076507568
User :  178
0.6581623554229736
User :  196
0.5029683113098145
User :  222
0.43993067741394043
User :  315

User :  354
0.248856782913208
User :  453
0.4457437992095947
User :  184
0.2079029083251953
User :  357
0.1310865879058838
User :  53
0.24129939079284668
User :  250
0.7871179580688477
User :  531
0.7415742874145508
User :  174
0.09160304069519043
User :  652
1.6386737823486328
User :  323
0.10693836212158203
User :  462
0.2788259983062744
User :  246
0.11793184280395508
User :  367
1.169496774673462
User :  180
0.09893345832824707
User :  218
0.2608375549316406
User :  230
0.5027234554290771
User :  612
0.37432003021240234
User :  60
0.3008260726928711
User :  348
0.27573585510253906
User :  393
0.261960506439209
User :  644
0.18790555000305176
User :  12
0.30481982231140137
User :  189
0.9140956401824951
User :  215
0.2808382511138916
User :  368
0.10094165802001953
User :  645
0.12991094589233398
User :  515
0.15491056442260742
User :  107
0.13992023468017578
User :  112
0.08696436882019043
User :  181
0.11991596221923828
User :  208
0.2234797477722168
User :  274
0.1858930587768554

In [29]:
global_mean_rating = np.average(train_df.rating.values)

In [30]:
global_mean_rating

3.5399152683169963

In [31]:
test_df1 = test_df1.fillna(global_mean_rating)

### Performance Evaluation on the training data

In [32]:
# The actual rating of the movies by the user
y_test = test_df1['rating'].values
# The predicted rating of the movies
y_test_pred = test_df1['prediction'].values

In [33]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import mean_absolute_error
# Root Mean Square Error of the model
rms = round(sqrt(mean_squared_error(y_test, y_test_pred )), 2)
print('Root Mean Square Error:',rms)
# Mean Absolute Error of the model
mse = round(mean_absolute_error(y_test, y_test_pred), 2)
print('Mean Absolute Error:',mse)

Root Mean Square Error: 1.06
Mean Absolute Error: 0.85


In [34]:
test_df1.to_csv("train_predict_content_df.csv", index=False)